In [1]:
from sklearn.datasets import load_iris
import sagemaker
import pandas as  pd

In [2]:
iris = load_iris(as_frame=True)["data"]

In [3]:
iris.to_csv("predictors.csv", 
            header=False, 
            index=False
           )

In [4]:
target = load_iris(as_frame=True)["target"]

In [5]:
data = pd.concat([target, iris], axis=1)

In [6]:
data.to_csv("data.csv", 
            header=False, 
            index=False
           )

In [7]:
session = sagemaker.Session()

In [8]:
region_name = session.boto_region_name

In [10]:
container_image = sagemaker.image_uris.retrieve(
    framework="xgboost", 
    region=region_name, 
    version="latest")
container_image

'811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest'

In [11]:
bucket = session.default_bucket()

In [12]:
uploaded_data = session.upload_data(path="data.csv", bucket=bucket, key_prefix="training")

In [13]:
uploaded_data

's3://sagemaker-us-east-1-885248014373/training/data.csv'

In [11]:
input_data = sagemaker.inputs.TrainingInput(s3_data=uploaded_data,
                                            content_type="text/csv")

In [12]:
role = "arn:aws:iam::885248014373:role/service-role/AmazonSageMaker-ExecutionRole-20210305T230941"

In [13]:
xgboost = sagemaker.estimator.Estimator(
    image_uri=container_image,
    role=role,
    instance_type="ml.m5.large", 
    instance_count=1,
    output_path=f"s3://{bucket}/output",
    sagemaker_session=session)   

In [14]:
xgboost.set_hyperparameters(num_round=5, max_depth=5)

In [15]:
xgboost.fit({"train": input_data}) 

2021-06-28 23:08:15 Starting - Starting the training job...
2021-06-28 23:08:17 Starting - Launching requested ML instancesProfilerReport-1624921694: InProgress
......
2021-06-28 23:09:52 Starting - Preparing the instances for training......
2021-06-28 23:10:59 Downloading - Downloading input data......
2021-06-28 23:12:15 Training - Training image download completed. Training in progress.
2021-06-28 23:12:15 Uploading - Uploading generated training model
2021-06-28 23:12:15 Completed - Training job completed
Arguments: train
[2021-06-28:23:12:02:INFO] Running standalone xgboost training.
[2021-06-28:23:12:02:INFO] Path /opt/ml/input/data/validation does not exist!
[2021-06-28:23:12:02:INFO] File size need to be processed in the node: 0.0mb. Available memory size in the node: 87.62mb
[2021-06-28:23:12:02:INFO] Determined delimiter of CSV input is ','
[23:12:02] S3DistributionType set as FullyReplicated
[23:12:02] 150x4 matrix with 600 entries loaded from /opt/ml/input/data/train?format

In [20]:
job_name = xgboost.latest_training_job.job_name
job_name

'xgboost-2021-06-28-23-08-14-913'

In [21]:
xgboost.model_data

's3://sagemaker-us-east-1-885248014373/output/xgboost-2021-06-28-23-08-14-913/output/model.tar.gz'

In [23]:
import boto3

In [24]:
runtime = boto3.client("sagemaker-runtime")

In [38]:
response = runtime.invoke_endpoint(
    EndpointName="Endpoint-soojgABWRArG",
    Body=dados)

In [39]:
response["Body"].read().decode("utf-8")

'0.08762577176094055,0.08762577176094055,0.08762577176094055,0.08762577176094055,0.08762577176094055,0.08762577176094055,0.08762577176094055,0.08762577176094055,0.08762577176094055,0.08762577176094055,0.08762577176094055,0.08762577176094055,0.08762577176094055,0.08762577176094055,0.08762577176094055,0.08762577176094055,0.08762577176094055,0.08762577176094055,0.08762577176094055,0.08762577176094055,0.08762577176094055,0.08762577176094055,0.08762577176094055,0.08762577176094055,0.08762577176094055,0.08762577176094055,0.08762577176094055,0.08762577176094055,0.08762577176094055,0.08762577176094055,0.08762577176094055,0.08762577176094055,0.08762577176094055,0.08762577176094055,0.08762577176094055,0.08762577176094055,0.08762577176094055,0.08762577176094055,0.08762577176094055,0.08762577176094055,0.08762577176094055,0.08762577176094055,0.08762577176094055,0.08762577176094055,0.08762577176094055,0.08762577176094055,0.08762577176094055,0.08762577176094055,0.08762577176094055,0.08762577176094055

In [28]:
with open("predictors.csv", "r") as f:
    dados = f.read()